##### Copyright 2022 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Otimizadores com APIs Core

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/core/optimizers_core"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/guide/core/optimizers_core.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a> </td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/guide/core/optimizers_core.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/guide/core/optimizers_core.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a> </td>
</table>

## Introdução

Este notebook apresenta o processo de criação de otimizadores personalizados com as [APIs de baixo nível do TensorFlow Core](https://www.tensorflow.org/guide/core). Veja a [Visão geral das APIs Core](https://www.tensorflow.org/guide/core) para saber mais sobre o TensorFlow Core e seus casos de uso pretendidos.

O módulo de [otimizadores Keras](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) é o kit de ferramentas de otimização recomendado para muitos propósitos de treinamento geral. Ele inclui uma variedade de otimizadores pré-construídos, bem como uma funcionalidade de subclassificação que pode ser personalizada. Os otimizadores Keras também são compatíveis com camadas, modelos e loops de treinamento personalizados criados com as APIs Core. Esses otimizadores pré-construídos e personalizáveis ​​são adequados para a maioria dos casos, mas as APIs Core permitem controle total sobre o processo de otimização. Por exemplo, técnicas como Sharpness-Aware Minimization (SAM) exigem que o modelo e o otimizador sejam acoplados, o que não se encaixa na definição tradicional de otimizadores de ML. Este guia percorre o processo de criação de otimizadores personalizados do zero usando as APIs Core, dando a você o poder de controle total sobre a estrutura, a implementação e o comportamento de seus otimizadores.

## Visão geral dos otimizadores

Um otimizador é um algoritmo usado para minimizar uma função de perda em relação aos parâmetros treináveis ​​de um modelo. A técnica de otimização mais direta é o método do gradiente descendente, que atualiza os parâmetros de um modelo de forma iterativa, dando um passo na direção do maior declive de sua função de perda. O tamanho do passo é diretamente proporcional ao tamanho do gradiente, o que pode ser problemático quando o gradiente é muito grande ou muito pequeno. Existem muitos outros otimizadores baseados em gradiente, como Adam, Adagrad e RMSprop, que aproveitam várias propriedades matemáticas de gradientes para eficiência de memória e convergência rápida.

## Configuração

In [ ]:
import matplotlib
from matplotlib import pyplot as plt
# Preset Matplotlib figure sizes.
matplotlib.rcParams['figure.figsize'] = [9, 6]

In [ ]:
import tensorflow as tf
print(tf.__version__)
# set random seed for reproducible results 
tf.random.set_seed(22)

## Método do gradiente descendente

A classe básica do otimizador deve ter um método de inicialização e uma função para atualizar uma lista de variáveis ​​dada uma lista de gradientes. Comece implementando o otimizador básico do método do gradiente descendente que atualiza cada variável subtraindo seu gradiente escalado por uma taxa de aprendizado.

In [ ]:
class GradientDescent(tf.Module):

  def __init__(self, learning_rate=1e-3):
    # Initialize parameters
    self.learning_rate = learning_rate
    self.title = f"Gradient descent optimizer: learning rate={self.learning_rate}"

  def apply_gradients(self, grads, vars):
    # Update variables
    for grad, var in zip(grads, vars):
      var.assign_sub(self.learning_rate*grad)

Para testar esse otimizador, crie uma função de perda de exemplo para minimizar em relação a uma única variável, $x$. Calcule sua função de gradiente e solucione para obter seu valor de parâmetro de minimização:

$$L = 2x^4 + 3x^3 + 2$$

$$\frac{dL}{dx} = 8x^3 + 9x^2$$

$\frac{dL}{dx}$ é 0 em $x = 0$, que é um ponto de sela e em $x = - \frac{9}{8}$, que é o mínimo global. Portanto, a função de perda é otimizada em $x^\star = - \frac{9}{8}$.

In [ ]:
x_vals = tf.linspace(-2, 2, 201)
x_vals = tf.cast(x_vals, tf.float32)

def loss(x):
  return 2*(x**4) + 3*(x**3) + 2

def grad(f, x):
  with tf.GradientTape() as tape:
    tape.watch(x)
    result = f(x)
  return tape.gradient(result, x)

plt.plot(x_vals, loss(x_vals), c='k', label = "Loss function")
plt.plot(x_vals, grad(loss, x_vals), c='tab:blue', label = "Gradient function")
plt.plot(0, loss(0),  marker="o", c='g', label = "Inflection point")
plt.plot(-9/8, loss(-9/8),  marker="o", c='r', label = "Global minimum")
plt.legend()
plt.ylim(0,5)
plt.xlabel("x")
plt.ylabel("loss")
plt.title("Sample loss function and gradient");

Escreva uma função para testar a convergência de um otimizador com uma única função de perda variável. Assuma que a convergência foi alcançada quando o valor do parâmetro atualizado no timestep $t$ for o mesmo que seu valor no timestep $t-1$. Encerre o teste após um determinado número de iterações e também acompanhe qualquer explosão de gradientes durante o processo. Para realmente desafiar o algoritmo de otimização, inicialize mal o parâmetro. No exemplo acima, $x = 2$ é uma boa escolha, pois envolve um gradiente acentuado que também leva a um ponto de inflexão.

In [ ]:
def convergence_test(optimizer, loss_fn, grad_fn=grad, init_val=2., max_iters=2000):
  # Function for optimizer convergence test
  print(optimizer.title)
  print("-------------------------------")
  # Initializing variables and structures
  x_star = tf.Variable(init_val)
  param_path = []
  converged = False

  for iter in range(1, max_iters + 1):
    x_grad = grad_fn(loss_fn, x_star)

    # Case for exploding gradient
    if tf.math.is_nan(x_grad):
      print(f"Gradient exploded at iteration {iter}\n")
      return []

    # Updating the variable and storing its old-version
    x_old = x_star.numpy()
    optimizer.apply_gradients([x_grad], [x_star])
    param_path.append(x_star.numpy())

    # Checking for convergence
    if x_star == x_old:
      print(f"Converged in {iter} iterations\n")
      converged = True
      break
      
  # Print early termination message
  if not converged:
    print(f"Exceeded maximum of {max_iters} iterations. Test terminated.\n")
  return param_path

Teste a convergência do otimizador de método do gradiente descendente para as seguintes taxas de aprendizado: 1e-3, 1e-2, 1e-1

In [ ]:
param_map_gd = {}
learning_rates = [1e-3, 1e-2, 1e-1]
for learning_rate in learning_rates:
  param_map_gd[learning_rate] = (convergence_test(
      GradientDescent(learning_rate=learning_rate), loss_fn=loss))

Visualize o caminho dos parâmetros sobre um gráfico de contorno da função de perda.

In [ ]:
def viz_paths(param_map, x_vals, loss_fn, title, max_iters=2000):
  # Creating a controur plot of the loss function
  t_vals = tf.range(1., max_iters + 100.)
  t_grid, x_grid = tf.meshgrid(t_vals, x_vals)
  loss_grid = tf.math.log(loss_fn(x_grid))
  plt.pcolormesh(t_vals, x_vals, loss_grid, vmin=0, shading='nearest')
  colors = ['r', 'w', 'c']
  # Plotting the parameter paths over the contour plot
  for i, learning_rate in enumerate(param_map):
    param_path = param_map[learning_rate]
    if len(param_path) > 0:
      x_star = param_path[-1]
      plt.plot(t_vals[:len(param_path)], param_path, c=colors[i])
      plt.plot(len(param_path), x_star, marker='o', c=colors[i], 
              label = f"x*: learning rate={learning_rate}")
  plt.xlabel("Iterations")
  plt.ylabel("Parameter value")
  plt.legend()
  plt.title(f"{title} parameter paths")

In [ ]:
viz_paths(param_map_gd, x_vals, loss, "Gradient descent")

O método do gradiente descendente parece ficar preso no ponto de inflexão ao usar taxas de aprendizado menores. Aumentar a taxa de aprendizado pode encorajar um movimento mais rápido em torno da região do platô devido a um tamanho de passo maior; no entanto, isso implica no risco de se ter gradientes explosivos nas primeiras iterações, quando a função de perda é extremamente íngreme.

## Método do gradiente descendente com momento

O método do gradiente descendente com momento não apenas usa o gradiente para atualizar uma variável, mas também envolve a mudança na posição de uma variável com base em sua atualização anterior. O parâmetro de momento determina o nível de influência que a atualização no instante $t-1$ tem sobre a atualização no instante $t$. Acumular o momento ajuda a mover as variáveis para ​​além das regiões de platô mais rapidamente do que ocorre com o método do gradiente descendente básico. A regra de atualização do momento é a seguinte:

$$\Delta_x^{[t]} = lr \cdot L^\prime(x^{[t-1]}) + p \cdot \Delta_x^{[t-1]}$$

$$x^{[t]} = x^{[t-1]} - \Delta_x^{[t]}$$

onde

- $x$: a variável que está sendo otimizada
- $\Delta_x$: mudança em $x$
- $lr$: taxa de aprendizagem
- $L^\prime(x)$: gradiente da função de perda em relação a x
- $p$: parâmetro de momento

In [ ]:
class Momentum(tf.Module):

  def __init__(self, learning_rate=1e-3, momentum=0.7):
    # Initialize parameters
    self.learning_rate = learning_rate
    self.momentum = momentum
    self.change = 0.
    self.title = f"Gradient descent optimizer: learning rate={self.learning_rate}"

  def apply_gradients(self, grads, vars):
    # Update variables 
    for grad, var in zip(grads, vars):
      curr_change = self.learning_rate*grad + self.momentum*self.change
      var.assign_sub(curr_change)
      self.change = curr_change

Teste a convergência do otimizador de momento para as seguintes taxas de aprendizado: 1e-3, 1e-2, 1e-1

In [ ]:
param_map_mtm = {}
learning_rates = [1e-3, 1e-2, 1e-1]
for learning_rate in learning_rates:
  param_map_mtm[learning_rate] = (convergence_test(
      Momentum(learning_rate=learning_rate),
      loss_fn=loss, grad_fn=grad))

Visualize o caminho dos parâmetros sobre um gráfico de contorno da função de perda.

In [ ]:
viz_paths(param_map_mtm, x_vals, loss, "Momentum")

## Estimativa do momento adaptativo (Adam)

O algoritmo Estimativa do Momento Adaptativo (Adaptive Moment Estimation - Adam) é uma técnica de otimização eficiente e altamente generalizável que utiliza duas metodologias principais método do gradiente descendente: momento e propagação do quadrado médio da raiz (Root Mean Square Propagation - RMSP). O momento ajuda a acelerar a descida do gradiente usando o primeiro momento (soma dos gradientes) juntamente com um parâmetro de decaimento. O RMSP é semelhante; no entanto, ele aproveita o segundo momento (soma dos gradientes ao quadrado).

O algoritmo Adam combina o primeiro e o segundo momento para fornecer uma regra de atualização mais generalizável. O sinal de uma variável, $x$, pode ser determinado calculando $\frac{x}{\sqrt{x^2}}$. O otimizador Adam usa esse resultado para calcular uma etapa de atualização que é efetivamente um sinal suavizado. Em vez de calcular $\frac{x}{\sqrt{x^2}}$, o otimizador calcula uma versão suavizada de $x$ (primeiro momento) e $x^2$ (segundo momento) para cada atualização de variável.


**Algoritmo Adam**

$\beta_1 \gets 0.9 ; \triangleright \text{literature value}$

$\beta_2 \gets 0.999 ; \triangleright \text{literature value}$

$lr \gets \text{1e-3} ; \triangleright \text{configurable learning rate}$

$\epsilon \gets \text{1e-7} ; \triangleright \text{prevents divide by 0 error}$

$V_{dv} \gets \vec {\underset{n\times1}{0}} ;\triangleright \text{stores momentum updates for each variable}$

$S_{dv} \gets \vec {\underset{n\times1}{0}} ; \triangleright \text{stores RMSP updates for each variable}$

$t \gets 1$

$\text{On iteration } t:$

$;;;; \text{For} (\frac{dL}{dv}, v) \text{ in gradient variable pairs}:$

$;;;;;;;; V_{dv_i} = \beta_1V_{dv_i} + (1 - \beta_1)\frac{dL}{dv} ; \triangleright \text{momentum update}$

$;;;;;;;; S_{dv_i} = \beta_2V_{dv_i} + (1 - \beta_2)(\frac{dL}{dv})^2 ; \triangleright \text{RMSP update}$

$;;;;;;;; v_{dv}^{bc} = \frac{V_{dv_i}}{(1-\beta_1)^t} ; \triangleright \text{momentum bias correction}$

$;;;;;;;; s_{dv}^{bc} = \frac{S_{dv_i}}{(1-\beta_2)^t} ; \triangleright \text{RMSP bias correction}$

$;;;;;;;; v = v - lr\frac{v_{dv}^{bc}}{\sqrt{s_{dv}^{bc}} + \epsilon} ; \triangleright \text{parameter update}$

$;;;;;;;; t = t + 1$

**Fim do algoritmo**

Dado que $V_{dv}$ e $S_{dv}$ são inicializados com 0 e que $\beta_1$ e $\beta_2$ são próximos de 1, as atualizações de momento e RMSP tentem naturalmente a 0; portanto, as variáveis ​​podem se beneficiar da correção de bias. A correção de bias também ajuda a controlar a oscilação dos pesos conforme eles se aproximam do mínimo global.

In [ ]:
class Adam(tf.Module):
  
    def __init__(self, learning_rate=1e-3, beta_1=0.9, beta_2=0.999, ep=1e-7):
      # Initialize the Adam parameters
      self.beta_1 = beta_1
      self.beta_2 = beta_2
      self.learning_rate = learning_rate
      self.ep = ep
      self.t = 1.
      self.v_dvar, self.s_dvar = [], []
      self.title = f"Adam: learning rate={self.learning_rate}"
      self.built = False

    def apply_gradients(self, grads, vars):
      # Set up moment and RMSprop slots for each variable on the first call
      if not self.built:
        for var in vars:
          v = tf.Variable(tf.zeros(shape=var.shape))
          s = tf.Variable(tf.zeros(shape=var.shape))
          self.v_dvar.append(v)
          self.s_dvar.append(s)
        self.built = True
      # Perform Adam updates
      for i, (d_var, var) in enumerate(zip(grads, vars)):
        # Moment calculation
        self.v_dvar[i] = self.beta_1*self.v_dvar[i] + (1-self.beta_1)*d_var
        # RMSprop calculation
        self.s_dvar[i] = self.beta_2*self.s_dvar[i] + (1-self.beta_2)*tf.square(d_var)
        # Bias correction
        v_dvar_bc = self.v_dvar[i]/(1-(self.beta_1**self.t))
        s_dvar_bc = self.s_dvar[i]/(1-(self.beta_2**self.t))
        # Update model variables
        var.assign_sub(self.learning_rate*(v_dvar_bc/(tf.sqrt(s_dvar_bc) + self.ep)))
      # Increment the iteration counter
      self.t += 1.

Teste o desempenho do otimizador Adam com as mesmas taxas de aprendizado usadas com os exemplos de método do gradiente descendente. 

In [ ]:
param_map_adam = {}
learning_rates = [1e-3, 1e-2, 1e-1]
for learning_rate in learning_rates:
  param_map_adam[learning_rate] = (convergence_test(
      Adam(learning_rate=learning_rate), loss_fn=loss))

Visualize o caminho dos parâmetros sobre um gráfico de contorno da função de perda.

In [ ]:
viz_paths(param_map_adam, x_vals, loss, "Adam")

Neste exemplo específico, o otimizador Adam tem convergência mais lenta em comparação com o método do gradiente descendente tradicional ao usar taxas de aprendizado pequenas. No entanto, o algoritmo move-se com sucesso além da região do platô e converge para o mínimo global quando uma taxa de aprendizado maior. Gradientes explosivos não são mais um problema devido ao redimensionamento dinâmico das taxas de aprendizado de Adam ao encontrar grandes gradientes.

## Conclusão

Este notebook apresentou os fundamentos da criação e comparação de otimizadores com as [APIs Core do TensorFlow](https://www.tensorflow.org/guide/core). Embora otimizadores pré-construídos como Adam sejam generalizáveis, eles nem sempre são a melhor alternativa para todo modelo ou dataset. Ter um controle refinado sobre o processo de otimização pode ajudar a simplificar os workflows de treinamento em ML e melhorar o desempenho geral. Consulte a documentação a seguir para mais exemplos de otimizadores personalizados:

- Este otimizador Adam é usado no tutorial [Perceptrons multicamada](https://www.tensorflow.org/guide/core/mlp_core) e no [Treinamento distribuído]()
- O [Model Garden](https://blog.tensorflow.org/2020/03/introducing-model-garden-for-tensorflow-2.html) tem uma variedade de [otimizadores personalizados](https://github.com/tensorflow/models/tree/master/official/modeling/optimization) escritos com as APIs Core.
